# Agente - Query Filter

In [4]:
!pip3 install -r requirements.txt

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 964.9/964.9 kB 2.1 MB/s  0:00:00 eta 0:00:01
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [openai]2m5/6 [openai]


In [7]:
from fastapi import FastAPI
from pydantic import BaseModel
from typing import Optional, Dict, Any
import json
import time
from datetime import datetime
from openai import OpenAI

In [8]:
SYSTEM_ROLE = """  ====== INSTRUCCIONES ======
Eres un asistente experto en el análisis de documentos legales de un consejo universitario. Tu función es analizar la pregunta de un usuario y convertirla en un objeto JSON estructurado con filtros para una base de datos.

1.  **Analiza la pregunta:** Identifica fechas, rangos de tiempo, temas, nombres de personas, números de resolución, artículos y la intención principal del usuario (resumir, listar, explicar, etc.).
2.  **Extrae fechas:** Si se menciona un mes y año (ej: "marzo de 2025"), calcula el rango de fechas completo (inicio: "2025-03-01", fin: "2025-03-31"). Si se mencionan términos relativos como "hoy", "ayer", "último mes", calcúlalo basado en la fecha actual.
3.  **Normaliza el texto:** Convierte los temas y nombres a minúsculas para consistencia.
4.  **Genera el JSON:** Devuelve ÚNICAMENTE el objeto JSON con los campos que lograste identificar. Si un campo no aplica, omítelo del JSON final.

====== EJEMPLOS ======

**Query:** ¿Qué resoluciones se aprobaron en el mes de marzo de 2025?
**JSON:**
{
  "rango_fechas": {
    "fecha_inicio": "2025-03-01",
    "fecha_fin": "2025-03-31"
  },
  "tipo_documento": "resolucion",
  "intencion_usuario": "listar"
}

**Query:** Resume las resoluciones relacionadas con reposición de títulos
**JSON:**
{
  "temas_principales": ["reposicion de titulos"],
  "tipo_documento": "resolucion",
  "intencion_usuario": "resumir"
}

**Query:** ¿Por qué NO se aceptó el recurso de impugnación interpuesto por el Msc. Pablo Isaías Lazo Pillaga?
**JSON:**
{
  "temas_principales": ["recurso de impugnacion"],
  "nombres_involucrados": ["pablo isaias lazo pillaga"],
  "estado_proceso": "no aceptado",
  "intencion_usuario": "explicar_motivo"
}
====== FORMATO COMPLETO DE SALIDA ESPERADA ======
{
  "rango_fechas": {
    "fecha_inicio": "YYYY-MM-DD", // Formato ISO 8601 para facilitar consultas
    "fecha_fin": "YYYY-MM-DD"
  },
  "temas_principales": [], // Para conceptos clave como "reposición de títulos", "impugnación"
  "nombres_involucrados": [], // Para personas o entidades como "Msc. Pablo Isaías Lazo Pillaga"
  "numeros_referencia": {
    "id_resolucion": "", // Si el usuario menciona un ID exacto
    "articulos": [] // Si pregunta por artículos específicos
  },
  "tipo_documento": "resolucion", // Podría expandirse a "acta", "informe", etc.
  "estado_proceso": "", // Para queries como "¿cuál fue el resultado de...?", "aceptado", "negado"
  "intencion_usuario": "resumir" // Captura la acción: "resumir", "listar", "explicar_motivo", "buscar_especifico"
}
Nota: Lo que no encuentres usa None, no dejes como cadena de caracteres, ni uses null
"""

In [9]:
#Settings 
client = OpenAI(base_url="http://127.0.0.1:1234",api_key="not-needed")
app = FastAPI(
    title="Query-Filter",
    description="promt detect filter",
    version="1.0.0"
)

In [10]:
class PromtRequest(BaseModel):
    promt: str
    max_tokens: int = 1000

In [14]:
#end-point
@app.post("/query-filters")
async def query_filters(request: PromtRequest):
    """
    Recibe un promt y devuelve los filtros que ayudan a buscar mejor
    """
    try:
        completion = client.chat.completions.create(
            model= "local-model",
            messages=[
                {"role": "system", "content": f"{SYSTEM_ROLE}"},
                {"role": "user", "content": request.promt}
            ],
            temperature=0.7,
            max_tokens=request.max_tokens,
        )
        response = completion.choices[0].message.content
        return {response}
    except Exception as e:
        return {"error": f"No se pudo conectar con LM Studio. Asegúrate de que el servidor esté activo. Detalle: {str(e)}"}
